In [1]:
!pip install nbimporter
!pip install nbformat

In [2]:
import pandas as pd
import sys
import os

In [19]:
#import in previously cleaned dataframes
event_df = pd.read_csv(r"C:\Users\Miranda\.ipython\CapstoneCoding\Medical_Conditionsta\event_df.csv")
conditon_df = pd.read_csv(r"C:\Users\Miranda\.ipython\CapstoneCoding\Medical_Conditionsta\condition_df.csv")

In [20]:
#read in hospital data
file_path = r"C:\Users\Miranda\.ipython\CapstoneCoding\Hospital_Inpatientdta\Hospital_Inpatient.dta"
hospital_df = pd.read_stata(file_path)
print(hospital_df.head(3))

      DUID  PID    DUPERSID           EVNTIDX    EVENTRN          ERHEVIDX  \
0  2790023  102  2790023102  2790023102002001  3 ROUND 3                -1   
1  2790046  103  2790046103  2790046103004701  3 ROUND 3  2790046103004801   
2  2790046  103  2790046103  2790046103004901  3 ROUND 3  2790046103005001   

  FFEEIDX        PANEL         MPCDATA IPBEGYR  ... IPDOF23X IPDSL23X  \
0      -1  27 PANEL 27  1 HAS MPC DATA    2022  ...      0.0      0.0   
1      -1  27 PANEL 27   2 NO MPC DATA    2023  ...      0.0      0.0   
2      -1  27 PANEL 27  1 HAS MPC DATA    2023  ...      0.0      0.0   

  IPDWC23X  IPDOT23X IPDXP23X IPDTC23X              IMPFLAG      PERWT23F  \
0      0.0       0.0  1126.52  18923.0  2 COMPLETE MPC DATA  19699.576310   
1      0.0       0.0   428.95   1464.0  4 PARTIALLY IMPUTED   9131.375404   
2      0.0       0.0   428.95   1464.0  2 COMPLETE MPC DATA   9131.375404   

  VARSTR VARPSU  
0   2037      1  
1   2055      3  
2   2055      3  

[3 rows x 50

In [21]:
#check columns and dtype
hospital_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 50 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   DUID      1830 non-null   int32   
 1   PID       1830 non-null   int16   
 2   DUPERSID  1830 non-null   object  
 3   EVNTIDX   1830 non-null   object  
 4   EVENTRN   1830 non-null   category
 5   ERHEVIDX  1830 non-null   object  
 6   FFEEIDX   1830 non-null   object  
 7   PANEL     1830 non-null   category
 8   MPCDATA   1830 non-null   category
 9   IPBEGYR   1830 non-null   category
 10  IPBEGMM   1830 non-null   category
 11  IPENDYR   1830 non-null   category
 12  IPENDMM   1830 non-null   category
 13  NUMNIGHX  1830 non-null   int16   
 14  EMERROOM  1830 non-null   category
 15  SPECCOND  1830 non-null   category
 16  RSNINHOS  1830 non-null   category
 17  ANYOPER   1830 non-null   category
 18  DSCHPMED  1830 non-null   category
 19  FFIPTYPE  1830 non-null   category
 20  IPXP23X 

In [22]:
#create df with columns of interest
hospital_df = hospital_df[['DUPERSID', 'EVNTIDX', 'IPENDMM', 'IPDMR23X','IPDMD23X','IPDPV23X',
                  'IPFMD23X','IPFMR23X','IPFPV23X', 'EVENTRN']].copy()
hospital_df.head(3)


,DUPERSID,EVNTIDX,IPENDMM,IPDMR23X,IPDMD23X,IPDPV23X,IPFMD23X,IPFMR23X,IPFPV23X,EVENTRN
0,2790023102,2790023102002001,1,1126.52,0.0,0.00,0.0,24094.55,0.00,3 ROUND 3
1,2790046103,2790046103004701,2,0.00,0.0,428.95,0.0,0.00,12097.77,3 ROUND 3
2,2790046103,2790046103004901,2,0.00,0.0,428.95,0.0,0.00,9672.40,3 ROUND 3


In [23]:
#change column names from codebook variables
hospital_df.rename(columns={'DUPERSID': 'person_id', 'EVNTIDX': 'visit_id', 'IPENDMM': 'month', 'IPDMR23X': 'dr_medicare',
                            'IPDMD23X': 'dr_medicaid','IPDPV23X': 'dr_private',
                            'IPFMD23X': 'f_medicaid','IPFMR23X': 'f_medicare','IPFPV23X': 'f_private', 'EVENTRN': 'round'}, 
                            inplace=True)
hospital_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   person_id    1830 non-null   object  
 1   visit_id     1830 non-null   object  
 2   month        1830 non-null   category
 3   dr_medicare  1830 non-null   float64 
 4   dr_medicaid  1830 non-null   float64 
 5   dr_private   1830 non-null   float64 
 6   f_medicaid   1830 non-null   float64 
 7   f_medicare   1830 non-null   float64 
 8   f_private    1830 non-null   float64 
 9   round        1830 non-null   category
dtypes: category(2), float64(6), object(2)
memory usage: 118.9+ KB


In [24]:
#sum dr and facility payments to one column for medicare, medicaid, private
hospital_df['medicaid']= hospital_df['dr_medicaid'] + hospital_df['f_medicaid']
hospital_df['medicare']= hospital_df['dr_medicare'] + hospital_df['f_medicare']
hospital_df['private'] = hospital_df['dr_private'] + hospital_df['f_private']

#drop old columns
hospital_df = hospital_df.drop(['dr_medicaid', 'f_medicaid', 'dr_medicare', 'f_medicare', 'dr_private', 'f_private'], axis=1)
hospital_df.head()

,person_id,visit_id,month,round,medicaid,medicare,private
0,2790023102,2790023102002001,1,3 ROUND 3,0.0,25221.07,0.00
1,2790046103,2790046103004701,2,3 ROUND 3,0.0,0.00,12526.72
2,2790046103,2790046103004901,2,3 ROUND 3,0.0,0.00,10101.35
3,2790050102,2790050102004601,5,4 ROUND 4,0.0,9826.38,0.00
4,2790050102,2790050102005401,10,5 ROUND 5,0.0,340.02,11026.60


In [25]:
#check category datatype in month column
hospital_df['month'].unique()

[1, 2, 5, 10, 12, ..., 3, 8, 7, 6, '-8 DON'T KNOW']
Length: 13
Categories (13, object): ['-8 DON'T KNOW' < 1 < 2 < 3 ... 9 < 10 < 11 < 12]

In [26]:
#drop invalid months
hospital_df = hospital_df.dropna(subset=['month'])

#change month datatype
#category label to string, coerce to NANs
hospital_df['month'] = pd.to_numeric(hospital_df['month'].astype(str), errors='coerce')
#drop Nans
hospital_df = hospital_df.dropna(subset=['month'])
#change datatype to int
hospital_df['month'] = hospital_df['month'].astype(int)

#change visit id to int
hospital_df['visit_id'] = hospital_df['visit_id'].astype(int)

hospital_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1819 entries, 0 to 1829
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   person_id  1819 non-null   object  
 1   visit_id   1819 non-null   int64   
 2   month      1819 non-null   int64   
 3   round      1819 non-null   category
 4   medicaid   1819 non-null   float64 
 5   medicare   1819 non-null   float64 
 6   private    1819 non-null   float64 
dtypes: category(1), float64(3), int64(2), object(1)
memory usage: 101.5+ KB


In [27]:
#import event_df and condition_df
event_df = pd.read_csv(r"C:\Users\Miranda\.ipython\CapstoneCoding\Medical_Conditionsta\event_df.csv")

condition_df = pd.read_csv(r"C:\Users\Miranda\.ipython\CapstoneCoding\Medical_Conditionsta\condition_df.csv")

In [28]:
#merge hospital df with event to link medical condtion
hospital_df = hospital_df.merge(event_df, on='visit_id', how='left')

hospital_df = hospital_df.merge(condition_df, on='condition_id', how='left')

#check new hospital dataframe
hospital_df.head()

,person_id,visit_id,month,round,medicaid,medicare,private,condition_id,ICD10
0,2790023102,2790023102002001,1,3 ROUND 3,0.0,25221.07,0.00,2.790023e+12,J18
1,2790046103,2790046103004701,2,3 ROUND 3,0.0,0.00,12526.72,2.790046e+12,J39
2,2790046103,2790046103004901,2,3 ROUND 3,0.0,0.00,10101.35,2.790046e+12,J39
3,2790050102,2790050102004601,5,4 ROUND 4,0.0,9826.38,0.00,2.790050e+12,-15
4,2790050102,2790050102005401,10,5 ROUND 5,0.0,340.02,11026.60,2.790050e+12,R41


In [29]:
#drop duplicates at visit and condition level together
hospital_df = hospital_df.drop_duplicates(subset=['visit_id','condition_id'])

#drop -15 from ICD10
hospital_df = hospital_df[hospital_df['ICD10'] != '-15']

In [30]:
#hospital dataframe
hospital_df.head()

,person_id,visit_id,month,round,medicaid,medicare,private,condition_id,ICD10
0,2790023102,2790023102002001,1,3 ROUND 3,0.0,25221.07,0.00,2.790023e+12,J18
1,2790046103,2790046103004701,2,3 ROUND 3,0.0,0.00,12526.72,2.790046e+12,J39
2,2790046103,2790046103004901,2,3 ROUND 3,0.0,0.00,10101.35,2.790046e+12,J39
4,2790050102,2790050102005401,10,5 ROUND 5,0.0,340.02,11026.60,2.790050e+12,R41
5,2790050102,2790050102005401,10,5 ROUND 5,0.0,340.02,11026.60,2.790050e+12,J18


In [31]:
#count conditions per visit(eliminate double payments for one visit with multiple conditions)
condition_count = hospital_df.groupby('visit_id')['condition_id'].nunique().reset_index(name='num_conditions')

#single visits by condition
single_condition = condition_count[condition_count['num_conditions'] ==1]['visit_id']
hospital_df = hospital_df[hospital_df['visit_id'].isin(single_condition)].copy()

In [32]:
hospital_df = hospital_df[['ICD10', 'medicaid','medicare', 'private', 'month']]
hospital_df.head(3)

,ICD10,medicaid,medicare,private,month
0,J18,0.0,25221.07,0.00,1
1,J39,0.0,0.00,12526.72,2
2,J39,0.0,0.00,10101.35,2


In [33]:
#save hospital dataframe to csv file for Tableau
hospital_df.to_csv(r"C:\Users\Miranda\.ipython\CapstoneCoding\dataframes_csv_format\hospital_df.csv", index=False)